# pySDC in a nutshell

pySDC = Python + spectral deferred correction (SDC) is a prototyping code for all things SDC, including PFASST.
It was started by Robert Speck about 10 years ago and is now primarily developed in Jülich and Hamburg.
The goal is to make SDC related research as accessible as possible.
It is very modular and abstract, which allows to focus only on small parts of the bigger SDC puzzle.
Furthermore, all methods are implemented in serial as well as in parallal to allow both easy developping as well as measuring performance.

In this notebook, we will briefly look into SDC and pySDC.

## Spectral deferred correction (SDC)
SDC is a method for the numerical integration of initial value problems of the sort $$u(\Delta t) = \int_0^{\Delta t} f(u) dt + u(0).$$
For the numerical treatment, we start with discretizing time into $M$ **collocation nodes** $0 \leq \tau_m \leq 1$.
Then, we interpolate $f$ $$f(u(t))\approx\sum_{m=1}^{M} l_m^\tau(t/\Delta t) f(u(\Delta t \tau_m))$$ using Lagrange polynomials $$l_j^{\tau}(t)=\frac{\prod_{i=1, i\neq j}^M (t-\tau_i)}{\prod_{i=1, i\neq j}^M (\tau_j-\tau_i)}.$$

Since only the Lagrange polynomials depend on $t$ in the interpolation, plugging into the initial value problem, we get $$u_m := u_0 + \Delta t\sum_{j=1}^M q_{mj}f(u(\Delta t\tau_j)) \approx u(\Delta t \tau_m),$$ with the **quadrature weights** $$q_{mj} = \int_{0}^{\tau_m} l_j^\tau(s) ds.$$

Collecting in vectors $\vec{u} = (u_m)^T$, we get $$(I - \Delta tQf)(\vec{u}) = \vec{u}_0,$$ with $f(\vec{u}) = (f(u_m))^T$ and $\vec{u_0} = (u_0)^T$.
This is called the **collocation problem**.
Before discussing how to solve this, let's look at a sample quadrature matrix.

The generation of quadrature matrices has been spun out from pySDC to a stand-alone repository [qmat](https://github.com/Parallel-in-Time/qmat).
If you use SDC outside of pySDC, please consider using qmat.

In [1]:
from qmat import genQCoeffs

nodes, _, Q = genQCoeffs("Collocation", nNodes=3, nodeType="LEGENDRE", quadType="RADAU-RIGHT")
print(f'{Q=}')

Q=array([[ 0.19681548, -0.06553543,  0.02377097],
       [ 0.39442431,  0.29207341, -0.04154875],
       [ 0.37640306,  0.51248583,  0.11111111]])


As you can see, $Q$ is densely populated.
Inverting $Q$ by itself is no problem, but if you are integrating a PDE, inverting $Qf$ can become prohibitively expensive.
Instead, we solve the collocation problem iteratively.

The simplest iterative scheme is standard Richardson iteration $$\vec{u}^{k+1} = \vec{u}_0 + \Delta tQf(\vec{u}^k),$$ but convergence is slow and limited.
Therefore, we precondition this iteration with $\Delta t Q_\Delta f$ to arrive at $$(I_M - \Delta t Q_\Delta f)(\vec{u}^{k+1}) =  \vec{u}_0 + \Delta t(Q-Q_\Delta )f(\vec{u}^k).$$

A typical choice for $Q_\Delta$ would be $Q_\Delta = U^T$, with $Q^T = LU$.
Again, we use qmat to obtain the preconditioner.

In [2]:
from qmat import genQDeltaCoeffs

QDelta = genQDeltaCoeffs("LU", Q=Q)
print(f'{QDelta=}')

QDelta=array([[0.19681548, 0.        , 0.        ],
       [0.39442431, 0.42340844, 0.        ],
       [0.37640306, 0.63782015, 0.2       ]])


Now you have all the ingredients to run vanilla SDC.
However, the advantage of SDC is that you can modify everything and design a crazy SDC scheme to fit your needs.
For instance, you can do splitting, multi-level, multi-step, quantum-AI, bank heist, ...

## pySDC
Before looking at some examples of what pySDC has been used for, we discuss its structure and how to set up a run.

We will first go through all core modules that you can / should configure.
Namely:
 - problem
 - sweeper
 - level
 - step
 - controller
 - hook
 - convergence controller
 - transfer class

The main hierarchy in pySDC is controller > step > level > sweeper > problem.
The rest are bells and whistles.

### Problem
At the bottom of the hierarchy are the problem objects.
These implement how to evaluate and invert $f$ for a given problem.
We will discuss in the next notebook how to implement problem classes.

### Sweeper
Next up in the hierarchy is the sweeper.
SDC iterations are often called sweeps due to their forward substitution nature, hence the name.
The sweeper takes care of the time-discretization.
It sets up the $Q$ and $Q_\Delta$ matrices and calls functions of the problem class during the actual time-stepping

### Level
SDC schemes are commonly designed with multiple levels similar to a multigrid scheme.
Corrections are computed more cheaply on coarser grids.
What ``coarser'' means is up for you to decide: Fewer collocation nodes, less spatial resolution, inexact approximation of inverse of $f$, ...
Each level administers one sweeper, which it calls to do the actual time-stepping on the level itself.

### Step
The step handles all of the levels.
It loops through them and calls the time-stepping on the individual levels.

### Controller
The pySDC controllers are the top-level objects that, well, control everything.
The controller can have multiple steps and loops through them, calling the integration, and communicating results.
Doing time-stepping on multiple steps in Gauß-Seidel or Jacobi fashion is a crucial ingredient of PFASST.

### Hook
Hooks are for recording data.
They are called by the controller in various places and have access to en entire level, so they can log pretty much anything.

### Convergence contollers
These have a very misleading name.
They are called by the controller just like the hooks, but can modify anything rather than log anything.

### Transfer class
These transfer the solution between different levels.


## Configuring pySDC
All of the building blocks above need to be configured for a run of pySDC.
We will now go through the process of configuring and running an Allen-Cahn equation.
We will define dictionaries of individual parameters and then gather all of them in one large dictionary which we pass to the controller.

In [3]:
from pySDC.implementations.sweeper_classes.generic_implicit import generic_implicit
from pySDC.implementations.controller_classes.controller_nonMPI import controller_nonMPI
from pySDC.implementations.hooks.log_solution import LogSolution
from pySDC.implementations.problem_classes.AllenCahn_2D_FD import allencahn_fullyimplicit
from pySDC.implementations.convergence_controller_classes.adaptivity import Adaptivity
from pySDC.implementations.transfer_classes.TransferMesh import mesh_to_mesh

# level and step parameters contain general parameters such as how many SDC iterations will be done
level_params = {}
level_params['dt'] = 1e-2
level_params['restol'] = 1e-5

step_params = {}
step_params['maxiter'] = 9

# the sweeper parameters describe the collocation problem and the preconditioner
sweeper_params = {}
sweeper_params['quad_type'] = 'RADAU-RIGHT'
sweeper_params['num_nodes'] = [2, 4]
sweeper_params['QI'] = ['LU', 'MIN-SR-S']

problem_params = {'nvars': [(128,) * 2, (64,) * 2], 'newton_maxiter': [5, 99]}

convergence_controllers = {}
# convergence_controllers[Adaptivity] = {'e_tol': 1e-4}  # for instance

# gather all parameters in one dictionary and add problem and sweeper class
description = {}
description['problem_class'] = allencahn_fullyimplicit
description['problem_params'] = problem_params
description['sweeper_class'] = generic_implicit
description['sweeper_params'] = sweeper_params
description['level_params'] = level_params
description['step_params'] = step_params
description['convergence_controllers'] = convergence_controllers
description['space_transfer_class'] = mesh_to_mesh

# more parameters for the controller
controller_params = {}
controller_params['logger_level'] = 15
controller_params['hook_class'] = [LogSolution]
controller_params['mssdc_jac'] = False

# setup controller
controller = controller_nonMPI(controller_params=controller_params, description=description, num_procs=2)

controller - INFO: Welcome to the one and only, really very astonishing and 87.3% bug free
                                 _____ _____   _____ 
                                / ____|  __ \ / ____|
                    _ __  _   _| (___ | |  | | |     
                   | '_ \| | | |\___ \| |  | | |     
                   | |_) | |_| |____) | |__| | |____ 
                   | .__/ \__, |_____/|_____/ \_____|
                   | |     __/ |                     
                   |_|    |___/                      
                                                     
controller - INFO: Setup overview (--> user-defined, -> dependency) -- BEGIN
controller - INFO: ----------------------------------------------------------------------------------------------------

Controller: <class 'pySDC.implementations.controller_classes.controller_nonMPI.controller_nonMPI'>
    all_to_done = False
    dump_setup = True
    fname = run_pid94048.log
--> hook_class = [<class 'pySDC.implementations.hoo

The above output shows the configuration that we just set up.
`-->` indicates that we set a parameter manually.
Other parameters are default values.

We chose a funny configuration with 2 collocation nodes, LU preconditioner, high resolution and inexact Newton solver on the fine level.
One the coarse level, we chose 4 collocation nodes, but diagonal preconditioner, lower spatial resultion, but we solve the non-linear problems to high accuracy.
Why did we do that?
To show off how easy it is to configure multi-level SDC in pySDC.
Finally, we chose 2 steps in parallel in Gauß-Seidel mode.

Was this a good idea?
Let's run this configuration and find out!

In [4]:
# get initial conditions
P = controller.MS[0].levels[0].prob  # observe the hierarchy
uinit = P.u_exact(t=0)

uend, stats = controller.run(u0=uinit, t0=0, Tend=2e-2)

hooks - INFO: Process  0 on time 0.000000 at stage       IT_COARSE: Level: 1 -- Iteration:  1 -- Sweep:  1 -- residual: 6.55548055e-02
hooks - INFO: Process  1 on time 0.010000 at stage       IT_COARSE: Level: 1 -- Iteration:  1 -- Sweep:  1 -- residual: 4.33590968e-01
hooks - INFO: Process  0 on time 0.000000 at stage         IT_FINE: Level: 0 -- Iteration:  1 -- Sweep:  1 -- residual: 5.69274516e-01
hooks - INFO: Process  1 on time 0.010000 at stage         IT_FINE: Level: 0 -- Iteration:  1 -- Sweep:  1 -- residual: 7.25755959e-01
hooks - INFO: Process  0 on time 0.000000 at stage       IT_COARSE: Level: 1 -- Iteration:  2 -- Sweep:  1 -- residual: 1.52277915e-02
hooks - INFO: Process  1 on time 0.010000 at stage       IT_COARSE: Level: 1 -- Iteration:  2 -- Sweep:  1 -- residual: 1.42393824e-01
hooks - INFO: Process  0 on time 0.000000 at stage         IT_FINE: Level: 0 -- Iteration:  2 -- Sweep:  1 -- residual: 5.95755213e-03
hooks - INFO: Process  1 on time 0.010000 at stage     

Well, we can already see that the residual did not reach the tolerance we set within the iteration limit.
So: no, this was not a smart idea.
Not all SDC variants are always useful.
But, if you actually have a think about what you're doing rather than just picking random configurations, SDC can be a quite powerful method!

For now, let's throw out the weird time coarsening and only do coarsening in space:

In [5]:
sweeper_params['QI'] = 'LU'
sweeper_params['num_nodes'] = 2
controller = controller_nonMPI(controller_params=controller_params, description=description, num_procs=2)
uend, stats = controller.run(u0=uinit, t0=0, Tend=2e-2)

controller - INFO: Welcome to the one and only, really very astonishing and 87.3% bug free
                                 _____ _____   _____ 
                                / ____|  __ \ / ____|
                    _ __  _   _| (___ | |  | | |     
                   | '_ \| | | |\___ \| |  | | |     
                   | |_) | |_| |____) | |__| | |____ 
                   | .__/ \__, |_____/|_____/ \_____|
                   | |     __/ |                     
                   |_|    |___/                      
                                                     
controller - INFO: Setup overview (--> user-defined, -> dependency) -- BEGIN
controller - INFO: ----------------------------------------------------------------------------------------------------

Controller: <class 'pySDC.implementations.controller_classes.controller_nonMPI.controller_nonMPI'>
    all_to_done = False
    dump_setup = True
    fname = run_pid94048.log
--> hook_class = [<class 'pySDC.implementations.hoo

Ah, much better.
Feel free to experiment further with the parameters to achieve even faster convergence.

## Some projects with pySDC
We will now look at a few representative projects done with pySDC

3D runs of Rayleigh-Benard convection and Gray-Scott

<img src="figs/3D_RBC_3.png" style="width:10cm;"/> <img src="figs/GS3D_000040.png" style="width:20cm;"/>

GPU implementation of 3D Gray-Scott scales to all of JUWELS booster when extending space scaling with diagonal SDC

<img src="figs/scaling_GS3D_time.png" style="width:20cm;"/>

Compare run time of various SDC configurations and compare preconditioner performance

<img src="figs/timings_SDC_variants_GrayScott.png" style="width:10cm;"/> <img src="figs/parallelSDC_preconditioner_vanderpol.png" style="width:10cm;"/>

Compare wall time of SDC against reference RK method

<img src="figs/wp-run_RBC-RK_comp-t-e_global_rel.png" style="width:10cm;"/>

Recover from faults in PFASST by interpolating from nearby steps

<img src="figs/ADVECTION_steps_vs_iteration_hf_7x7_INTERP.png" style="width:10cm;"/>

SDC maintains convergence order up to compression threshold when storing compressed data 

<img src="figs/compression_order_time_advection_d=1.00e-06_n=4_MPI=True.png" style="width:10cm;"/>

As you can see, pySDC is a flexible tool, capable of loads of things.
If you want to 
 - design a novel SDC scheme
 - solve the heat equation
 - count iterations
 - solve very complicated equations
 - measure wall time in actual HPC settings
 - investigate any SDC related idea
 - like your code tested

Then pySDC is the code for you!
Get in touch if you want to collaborate or need help with anything!